<a href="https://colab.research.google.com/github/desininja/DE1_practice/blob/4-project-etl/ETL_Spark_Job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=be487498638fbf3e66060d808a7fd1a8ed94071750f4b9d2759f71392c2db991
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL Project").master('local').getOrCreate()

sc = spark.sparkContext

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,BooleanType, DoubleType,LongType,FloatType


In [4]:
fileSchema = StructType([StructField('year',IntegerType(),True),
                        StructField('month',StringType(),True),
                         StructField('day',IntegerType(),True),
                         StructField('weekday',StringType(),True),
                         StructField('hour',IntegerType(),True),
                         StructField('atm_status',StringType(),True),
                         StructField('atm_id',StringType(),True),
                         StructField('atm_manufacturer',StringType(),True),
                         StructField('atm_location',StringType(),True),
                         StructField('atm_streetname',StringType(),True),
                         StructField('atm_street_number',IntegerType(),True),
                         StructField('atm_zipcode',IntegerType(),True),
                         StructField('atm_lat',DoubleType(),True),
                         StructField('atm_lon',DoubleType(),True),
                         StructField('currency',StringType(),True),
                         StructField('card_type',StringType(),True),
                         StructField('transaction_amount',IntegerType(),True),
                         StructField('service',StringType(),True),
                         StructField('message_code',StringType(),True),
                         StructField('message_text',StringType(),True),
                         StructField('weather_lat',DoubleType(),True),
                         StructField('weather_lon',DoubleType(),True),
                         StructField('weather_city_id',IntegerType(),True),
                         StructField('weather_city_name',StringType(),True),
                         StructField('temp',DoubleType(),True),
                         StructField('pressure',IntegerType(),True),
                         StructField('humidity',IntegerType(),True),
                         StructField('wind_speed',IntegerType(),True),
                         StructField('wind_deg',IntegerType(),True),
                         StructField('rain_3h',DoubleType(),True),
                         StructField('clouds_all',IntegerType(),True),
                         StructField('weather_id',IntegerType(),True),
                         StructField('weather_main',StringType(),True),
                         StructField('weather_description',StringType(),True)])


In [6]:
df1 = spark.read.csv('/content/atm_data_part1.csv',header=True,schema=fileSchema,sep=',')
df2 = spark.read.csv('/content/atm_data_part2.csv',header=True,schema=fileSchema,sep=',')

df = df1.union(df2)

In [7]:
df.show()

+----+-------+---+-------+----+----------+------+----------------+------------+--------------------+-----------------+-----------+-----------+-----------+--------+--------------------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+--------------------+-------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|      atm_streetname|atm_street_number|atm_zipcode|    atm_lat|    atm_lon|currency|           card_type|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|        weather_main|weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+--------------------+-----------------+-----------+-----------+-----------+--------+-------------

In [8]:
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

In [9]:
df.count()

9875

In [10]:
df.columns

['year',
 'month',
 'day',
 'weekday',
 'hour',
 'atm_status',
 'atm_id',
 'atm_manufacturer',
 'atm_location',
 'atm_streetname',
 'atm_street_number',
 'atm_zipcode',
 'atm_lat',
 'atm_lon',
 'currency',
 'card_type',
 'transaction_amount',
 'service',
 'message_code',
 'message_text',
 'weather_lat',
 'weather_lon',
 'weather_city_id',
 'weather_city_name',
 'temp',
 'pressure',
 'humidity',
 'wind_speed',
 'wind_deg',
 'rain_3h',
 'clouds_all',
 'weather_id',
 'weather_main',
 'weather_description']

In [18]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number

#res_df = emp_df.select("emp_name","dept","salary",F.row_number().over(Window.partitionBy().orderBy(res_df['emp_name'])).alias("index"))



# data = data.select(col("Name").alias("name"), col("askdaosdka").alias("age"))
# data.show()

In [20]:
#Location dimension - This is a very important dimension containing all the location data including
#location name, street name, street number, zip code and even the latitude and longitude. This
#information will be very important for solving problems related to the particular location at which
#a transaction took place and can help banks in things like pinpointing locations where ATMs where
#demand is higher as compared to other locations. Combined with weather data in the transaction table,
# this can be used to further do analysis such as how weather affects the demand at ATMs
#at a particular location.


dim_location = df.select(col('atm_location').alias('location'),col('atm_streetname').alias('street_name'),col('atm_street_number').alias('street_number'),col('atm_zipcode').alias('zipcode'),col('atm_lat').alias('lat'),col('atm_lon').alias('lon')).distinct()


In [22]:
dim_location = dim_location.select('location','street_name','street_number','zipcode','lat','lon',F.row_number().over(Window.partitionBy().orderBy(dim_location['lon'])).alias('location_id'))
dim_location.show()


+-------------------+-------------+-------------+-------+-----------+----------+-----------+
|           location|  street_name|street_number|zipcode|        lat|       lon|location_id|
+-------------------+-------------+-------------+-------+-----------+----------+-----------+
|            Sædding|  Tarphagevej|           59|   6710|55.49817127|8.40834784|          1|
|            Esbjerg| Strandbygade|           20|   6700|55.46817465|8.44026782|          2|
|          Holstebro|  Hostrupsvej|            6|   7500|56.37271162|8.62513609|          3|
|           Vinderup|   Søndergade|            5|   7830|56.48130757|  8.779161|          4|
|Nykøbing Mors Lobby|  Kirketorvet|            1|   7900|56.79501966|8.86027738|          5|
|      Nykøbing Mors|  Kirketorvet|            1|   7900|56.79501966|8.86027738|          6|
|           Glyngøre|     Færgevej|            1|   7870| 56.7623283|8.86671603|          7|
|              Durup|       Torvet|            4|   7870|56.74472535|8

In [23]:
dim_location.count()

105

In [24]:
#ATM dimension - This dimension will have the data related to the various ATMs
#present in the dataset along with the ATM number(ATM ID in the original dataset),
#ATM manufacturer and a reference to the ATM location and is very important for
#solving analytical queries related where ATM data will be used.



dim_atm = df.select(col('atm_location').alias('location'),
                         col('atm_streetname').alias('street_name'),
                         col('atm_street_number').alias('street_number'),
                         col('atm_zipcode').alias('zipcode'),
                         col('atm_lat').alias('lat'),
                         col('atm_lon').alias('lon'),
                         col("atm_id").alias("atm_number"),
                         col("atm_manufacturer").alias("manufacturer")).distinct()


dim_atm = dim_atm.join(dim_location, on=['location','street_name','street_number','zipcode','lat','lon'],how='left').select('atm_number','manufacturer','location_id')

dim_atm.show()

+----------+---------------+-----------+
|atm_number|   manufacturer|location_id|
+----------+---------------+-----------+
|        76|            NCR|         18|
|        86|            NCR|         99|
|        53|            NCR|         81|
|        57|            NCR|        100|
|        72|            NCR|         77|
|        70|Diebold Nixdorf|          3|
|        96|            NCR|         47|
|         6|            NCR|         32|
|        36|            NCR|         59|
|        64|            NCR|         37|
|        74|            NCR|         10|
|        98|            NCR|         79|
|        40|Diebold Nixdorf|         85|
|        62|Diebold Nixdorf|         68|
|        41|Diebold Nixdorf|         86|
|        78|Diebold Nixdorf|         90|
|        80|            NCR|         67|
|        59|Diebold Nixdorf|          5|
|        85|Diebold Nixdorf|        104|
|        31|            NCR|         91|
+----------+---------------+-----------+
only showing top

In [25]:
dim_atm = dim_atm.select('atm_number','manufacturer','location_id',F.row_number().over(Window.partitionBy().orderBy(dim_atm['atm_number'])).alias("atm_id"))
dim_atm.show()


+----------+---------------+-----------+------+
|atm_number|   manufacturer|location_id|atm_id|
+----------+---------------+-----------+------+
|         1|            NCR|         94|     1|
|        10|            NCR|         51|     2|
|       100|            NCR|         14|     3|
|       101|            NCR|         24|     4|
|       102|            NCR|         42|     5|
|       103|Diebold Nixdorf|         56|     6|
|       104|            NCR|         50|     7|
|       105|Diebold Nixdorf|         51|     8|
|       106|            NCR|         97|     9|
|       108|            NCR|         64|    10|
|        11|            NCR|         53|    11|
|        12|            NCR|         48|    12|
|        13|            NCR|         83|    13|
|        14|            NCR|         65|    14|
|        15|            NCR|         45|    15|
|        16|            NCR|         12|    16|
|        17|            NCR|         66|    17|
|        18|Diebold Nixdorf|         21|

In [13]:

from pyspark.sql.functions import *

In [27]:
#Date dimension - This is another very important dimension which is almost always present where data
#such as transactional data is being dealt with. This dimension includes fields such as the full date
#and time timestamp, year, month, day, hour as well as the weekday for a transaction. This all can help
#in analysing the transaction behaviour with respect to the time at which the transaction took place
#and also how the transaction activity varies between weekdays and weekends.

from pyspark.sql.functions import col, concat_ws

def month_to_numeric(month_name):
    # Extract the month name string from the Column object
    month_name_str = str(month_name)
    months = {
        "January": "01",
        "February": "02",
        "March": "03",
        "April": "04",
        "May": "05",
        "June": "06",
        "July": "07",
        "August": "08",
        "September": "09",
        "October": "10",
        "November": "11",
        "December": "12"
    }
    return months.get(month_name_str, "00")  # Return "00" for unknown months


month_to_numeric("January")



'01'

In [71]:
dim_date = df.select('year','month','day','hour','weekday').distinct()

In [77]:
res_date_df = dim_date.withColumn("full_date",from_unixtime(unix_timestamp(concat(dim_date.year.cast(StringType()),dim_date.month.cast(StringType()),lpad(dim_date.day.cast(StringType()),2,'0'),lpad(dim_date.hour.cast(StringType()),2,'0')),'yyyyMMMMddHH'),'yyyy-MM-dd HH:mm:ss'))

res_date_df.show()

+----+-------+---+----+-------+-------------------+
|year|  month|day|hour|weekday|          full_date|
+----+-------+---+----+-------+-------------------+
|2017|January|  1|   9| Sunday|2017-01-01 09:00:00|
|2017|January|  2|   8| Monday|2017-01-02 08:00:00|
|2017|January|  2|   2| Monday|2017-01-02 02:00:00|
|2017|January|  1|   7| Sunday|2017-01-01 07:00:00|
|2017|January|  1|   3| Sunday|2017-01-01 03:00:00|
|2017|January|  1|  20| Sunday|2017-01-01 20:00:00|
|2017|January|  1|  21| Sunday|2017-01-01 21:00:00|
|2017|January|  1|  17| Sunday|2017-01-01 17:00:00|
|2017|January|  2|   7| Monday|2017-01-02 07:00:00|
|2017|January|  1|   5| Sunday|2017-01-01 05:00:00|
|2017|January|  2|   9| Monday|2017-01-02 09:00:00|
|2017|January|  1|  15| Sunday|2017-01-01 15:00:00|
|2017|January|  2|   4| Monday|2017-01-02 04:00:00|
|2017|January|  2|   5| Monday|2017-01-02 05:00:00|
|2017|January|  1|  11| Sunday|2017-01-01 11:00:00|
|2017|January|  2|  10| Monday|2017-01-02 10:00:00|
|2017|Januar

In [78]:
dim_date = res_date_df.select("full_date","year","month","day","hour",
                                    "weekday",
                                    F.row_number().over(Window.partitionBy().orderBy(res_date_df['full_date'])).alias("date_id"))
#F.row_number().over(Window.partitionBy().orderBy(dim_atm['atm_number'])).alias("atm_id"))
dim_date.show()

+-------------------+----+-------+---+----+-------+-------+
|          full_date|year|  month|day|hour|weekday|date_id|
+-------------------+----+-------+---+----+-------+-------+
|2017-01-01 00:00:00|2017|January|  1|   0| Sunday|      1|
|2017-01-01 01:00:00|2017|January|  1|   1| Sunday|      2|
|2017-01-01 02:00:00|2017|January|  1|   2| Sunday|      3|
|2017-01-01 03:00:00|2017|January|  1|   3| Sunday|      4|
|2017-01-01 04:00:00|2017|January|  1|   4| Sunday|      5|
|2017-01-01 05:00:00|2017|January|  1|   5| Sunday|      6|
|2017-01-01 06:00:00|2017|January|  1|   6| Sunday|      7|
|2017-01-01 07:00:00|2017|January|  1|   7| Sunday|      8|
|2017-01-01 08:00:00|2017|January|  1|   8| Sunday|      9|
|2017-01-01 09:00:00|2017|January|  1|   9| Sunday|     10|
|2017-01-01 10:00:00|2017|January|  1|  10| Sunday|     11|
|2017-01-01 11:00:00|2017|January|  1|  11| Sunday|     12|
|2017-01-01 12:00:00|2017|January|  1|  12| Sunday|     13|
|2017-01-01 13:00:00|2017|January|  1|  

In [79]:
#Card type dimension - This dimension has the information about the particular card type with which
#a particular transaction took place. This can help in performing analysis on how the number of
#transactions varies with respect to each different card type.

dim_card_type = df.select('card_type').distinct()

dim_card_type = dim_card_type.select('card_type',F.row_number().over(Window.partitionBy().orderBy(dim_card_type['card_type'])).alias("card_id"))
dim_card_type.show()

+--------------------+-------+
|           card_type|card_id|
+--------------------+-------+
|              CIRRUS|      1|
|             Dankort|      2|
|     Dankort - on-us|      3|
|            Hævekort|      4|
|    Hævekort - on-us|      5|
|             Maestro|      6|
|          MasterCard|      7|
|  Mastercard - on-us|      8|
|                VISA|      9|
|        Visa Dankort|     10|
|Visa Dankort - on-us|     11|
|            VisaPlus|     12|
+--------------------+-------+



In [80]:
#Transaction fact - This is the actual fact table for the data set which contains all of the
#numerical data such as the currency of the transaction, service, transaction amount, message
#code and text as well as weather info such as description, weather id etc.


dim_atm.show(3)

+----------+------------+-----------+------+
|atm_number|manufacturer|location_id|atm_id|
+----------+------------+-----------+------+
|         1|         NCR|         94|     1|
|        10|         NCR|         51|     2|
|       100|         NCR|         14|     3|
+----------+------------+-----------+------+
only showing top 3 rows



In [81]:
dim_location.show(3)

+---------+------------+-------------+-------+-----------+----------+-----------+
| location| street_name|street_number|zipcode|        lat|       lon|location_id|
+---------+------------+-------------+-------+-----------+----------+-----------+
|  Sædding| Tarphagevej|           59|   6710|55.49817127|8.40834784|          1|
|  Esbjerg|Strandbygade|           20|   6700|55.46817465|8.44026782|          2|
|Holstebro| Hostrupsvej|            6|   7500|56.37271162|8.62513609|          3|
+---------+------------+-------------+-------+-----------+----------+-----------+
only showing top 3 rows



In [82]:
dim_date.show(3)

+-------------------+----+-------+---+----+-------+-------+
|          full_date|year|  month|day|hour|weekday|date_id|
+-------------------+----+-------+---+----+-------+-------+
|2017-01-01 00:00:00|2017|January|  1|   0| Sunday|      1|
|2017-01-01 01:00:00|2017|January|  1|   1| Sunday|      2|
|2017-01-01 02:00:00|2017|January|  1|   2| Sunday|      3|
+-------------------+----+-------+---+----+-------+-------+
only showing top 3 rows



In [83]:
dim_card_type.show(3)

+---------------+-------+
|      card_type|card_id|
+---------------+-------+
|         CIRRUS|      1|
|        Dankort|      2|
|Dankort - on-us|      3|
+---------------+-------+
only showing top 3 rows



In [87]:
stg1_df = df.join(dim_date, on=["year","month",'day',"hour","weekday"], how='left').select(*(df.columns+[dim_date.date_id])).drop(*['year','month','day','hour','weekday'])

In [88]:
stg1_df.show(1)

+----------+------+----------------+------------+--------------+-----------------+-----------+-----------+-----------+--------+----------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+
|atm_status|atm_id|atm_manufacturer|atm_location|atm_streetname|atm_street_number|atm_zipcode|    atm_lat|    atm_lon|currency| card_type|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|date_id|
+----------+------+----------------+------------+--------------+-----------------+-----------+-----------+-----------+--------+----------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+---

In [100]:


stg2_df = stg1_df.join(dim_card_type, on=['card_type'], how ='left').select(*(stg1_df.columns+[dim_card_type.card_id])).drop(*['card_type'])


In [101]:
stg2_df.show(2)
stg2_df.printSchema()

+----------+------+----------------+------------+--------------+-----------------+-----------+-----------+-----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+
|atm_status|atm_id|atm_manufacturer|atm_location|atm_streetname|atm_street_number|atm_zipcode|    atm_lat|    atm_lon|currency|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|date_id|card_id|
+----------+------+----------------+------------+--------------+-----------------+-----------+-----------+-----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+

In [105]:
# |location_id|
stg3_df = stg2_df.withColumnsRenamed({"atm_manufacturer":"manufacturer",
                                      "atm_location":"location",
                                      "atm_streetname":"street_name",
                                      "atm_street_number":"street_number",
                                      "atm_zipcode":"zipcode",
                                      "atm_lat":"lat",
                                      "atm_lon":"lon",
                                      "atm_id":"atm_number"})


stg3_df.show(3)

+----------+----------+------------+--------+-----------+-------------+-------+-----------+-----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+
|atm_status|atm_number|manufacturer|location|street_name|street_number|zipcode|        lat|        lon|currency|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|date_id|card_id|
+----------+----------+------------+--------+-----------+-------------+-------+-----------+-----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------

In [106]:
stg4_df = stg3_df.join(dim_location, on=["location","street_name","street_number","zipcode","lat","lon"], how='left').select(*(stg3_df.columns+[dim_location.location_id])).drop(*["location","street_name","street_number","zipcode","lat","lon"])

stg4_df.show(3)

+----------+----------+------------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+-----------+
|atm_status|atm_number|manufacturer|currency|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|date_id|card_id|location_id|
+----------+----------+------------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+-----------+
|    Active|         1|         NCR|     DKK|              NULL|   NULL|        NULL|   55.229919|   11.76092|  2616038.0

In [112]:
#|atm_number|manufacturer|location_id|atm_id|

stg5_df = stg4_df.join(dim_atm, on =['atm_number',"manufacturer","location_id"],how='left').select(*(stg4_df.columns+[dim_atm.atm_id])).drop(*['atm_number',"manufacturer"])

stg5_df.show(2)

+----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+-----------+------+
|atm_status|currency|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|date_id|card_id|location_id|atm_id|
+----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+-----------+------+
|    Active|     DKK|              NULL|   NULL|        NULL|   55.229919|   11.76092|  2616038.0|           NULL|           281.15|1014.0|      87|       7|       260|    

In [113]:
stg5_df.count()

9875

In [114]:
df.count()

9875

In [115]:

stg5_df.columns

['atm_status',
 'currency',
 'transaction_amount',
 'service',
 'message_code',
 'message_text',
 'weather_lat',
 'weather_lon',
 'weather_city_id',
 'weather_city_name',
 'temp',
 'pressure',
 'humidity',
 'wind_speed',
 'wind_deg',
 'rain_3h',
 'clouds_all',
 'weather_id',
 'weather_main',
 'weather_description',
 'date_id',
 'card_id',
 'location_id',
 'atm_id']

In [116]:
trans_fact = stg5_df.select('atm_status','currency','transaction_amount', 'service',
 'message_code', 'message_text', 'weather_lat', 'weather_lon',
 'weather_city_id', 'weather_city_name', 'temp', 'pressure',
 'humidity', 'wind_speed', 'wind_deg', 'rain_3h',
 'clouds_all', 'weather_id', 'weather_main', 'weather_description',
 'date_id', 'card_id', 'location_id', 'atm_id',F.row_number().over(Window.partitionBy().orderBy(stg5_df['atm_id'])).alias("trans_id"))

trans_fact.show(2)

+----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+-----------+------+--------+
|atm_status|currency|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|date_id|card_id|location_id|atm_id|trans_id|
+----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+-------+-------+-----------+------+--------+
|    Active|     DKK|              NULL|   NULL|        NULL|   55.229919|   11.76092|  2616038.0|           NULL|           281.15|1014.0|      

In [117]:
trans_fact.count()

9875

In [ ]:
trans_fact.coalesce(10).write.csv("address", mode="overwrite")

In [ ]:

dim_location.coalesce(1).write.csv("address", mode="overwrite")
dim_atm.coalesce(1).write.csv("address", mode="overwrite")
dim_date.coalesce(1).write.csv("address",mode="overwrite")
dim_card_type.coalese(1).write.csv("address",mode="overwrite")

In [119]:

# Writing DataFrame to CSV
trans_fact.write.csv("sample_data/transaction_fact_table.csv", header=True)
trans_fact.coalesce(10).write.csv("sample_data/transaction_fact_table_1.csv", mode="overwrite")


In [ ]:
dim_location.coalesce(1).write.csv("sample_data/dim_location.csv", mode="overwrite")
dim_atm.coalesce(1).write.csv("sample_data/dim_atm.csv", mode="overwrite")
dim_date.coalesce(1).write.csv("sample_data/dim_date.csv",mode="overwrite")
dim_card_type.coalese(1).write.csv("sample_data/dim_card_type.csv",mode="overwrite")